In [4]:
from rbc_parser import rbc_parser
from datetime import datetime, timedelta

# Задаем параметры запросы и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2021-01-01'
# dateTo = "2021-02-28"
dateTo = "2021-01-05"
offset = 0
limit = 100

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 100 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
# tbl = parser._get_search_table(param_dict,
#                                includeText = True) # Парсить текст статей
tbl = parser.get_articles2(param_dict,
                          count=10) # Парсить текст статей
print(len(tbl))
tbl.head()
len(tbl), tbl['text'][0][:100]

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.01.2021', 'dateTo': '05.01.2021', 'offset': '0', 'limit': '100', 'material': ''}
Parsing articles from  0
https://www.rbc.ru/v10/search/ajax/?project=rbcnews&category=TopRbcRu_economics&dateFrom=01.01.2021&dateTo=05.01.2021&offset=0&limit=100&query=РБК&material=


/home/misha-sh/rbc_parser.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(new_arcticles, ignore_index=True)


Parsing articles from  10
https://www.rbc.ru/v10/search/ajax/?project=rbcnews&category=TopRbcRu_economics&dateFrom=01.01.2021&dateTo=05.01.2021&offset=10&limit=100&query=РБК&material=
Finish
20


/home/misha-sh/rbc_parser.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(new_arcticles, ignore_index=True)


(20,
 'Минюст признал писателя Дмитрия Глуховского, автора романов «Метро 2033» и «Текст», и рэпера Oxxxymi')

In [5]:
# tokenize stopwords lemmatizer/stemmer(snowball)

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import pymorphy2
import string

ru_stopwords = set(stopwords.words("russian") + list(string.punctuation + '»' + '—' +  '«'))
only_words_tokenizer = RegexpTokenizer(r'\w+')
snowball = SnowballStemmer(language="russian")
morph = pymorphy2.MorphAnalyzer()

def tokenize(text):
    # text = only_words_tokenizer.tokenize(text) 
    text = word_tokenize(text, language="russian")
    res = []
    for word in text:
        if word not in ru_stopwords and len(word) > 1:
            res.append(word)
    return res

def keywords_calc(text):
    words = tokenize(text)
    res = {}
    for word in words:
        short = snowball.stem(word)
        if short not in res:
            normal = morph.parse(word)
            if len(normal) == 0:
                continue
            normal = normal[0]
            
            res[short] = { 
                "word": normal.normal_form if normal.score > 0.5 else word,
                "short": short,
                "count": 0
            }
        res[short]["count"] += 1
            
    return res

keywords_calc(tbl['text'][0][:200])


[nltk_data] Downloading package stopwords to /home/misha-
[nltk_data]     sh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/misha-sh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'минюст': {'word': 'Минюст', 'short': 'минюст', 'count': 1},
 'призна': {'word': 'признать', 'short': 'призна', 'count': 1},
 'писател': {'word': 'писатель', 'short': 'писател', 'count': 1},
 'дмитр': {'word': 'дмитрий', 'short': 'дмитр', 'count': 1},
 'глуховск': {'word': 'Глуховского', 'short': 'глуховск', 'count': 2},
 'автор': {'word': 'автор', 'short': 'автор', 'count': 1},
 'роман': {'word': 'роман', 'short': 'роман', 'count': 1},
 'метр': {'word': 'Метро', 'short': 'метр', 'count': 1},
 '2033': {'word': '2033', 'short': '2033', 'count': 1},
 'текст': {'word': 'текст', 'short': 'текст', 'count': 1},
 'рэпер': {'word': 'рэпер', 'short': 'рэпер', 'count': 1},
 'Oxxxymiron': {'word': 'oxxxymiron', 'short': 'Oxxxymiron', 'count': 1},
 'иноагент': {'word': 'иноагентами', 'short': 'иноагент', 'count': 1},
 'вмест': {'word': 'вместе', 'short': 'вмест', 'count': 1},
 'список': {'word': 'список', 'short': 'список', 'count': 1},
 'сми-иноагент': {'word': 'СМИ-иноагентов',
  'short': 'сми-

In [15]:
def keywords_sum(keyword_groups):
    res = {}
    for keywords in keyword_groups:
        for key in keywords:
            if key not in res:
                res[key] = keywords[key]
            else:
                res[key]["count"] += keywords[key]["count"]
    return res

def keywords_mean(keywords_groups):
    res = keywords_sum(keywords_groups)
    for key in res:
        res[key]["count"] /= len(keywords_groups)
    return res

def most_popular_keywords(keywords, count):
    res = []
    for key in keywords:
        res.append(keywords[key])
    res.sort(key=lambda k: -k["count"])
    return res[:count]


# left - right
def keywords_diff(left, right):
    res = left.copy()
    for key in right:
        if key not in res:
            res[key] = right[key] 
            res[key]["count"] = -res[key]["count"]
        else:
            res[key]["count"] -= right[key]["count"]
    return res

keyword_groups = [keywords_calc(i) for i in tbl['text'][:5]]
# keywords = keywords_sum(keyword_groups)
# most_popular_keywords(keywords, 5)

most_popular_keywords(keywords_diff(keyword_groups[0], keywords_mean(keyword_groups[1:])), 5)

[{'word': 'признать', 'short': 'призна', 'count': 4},
 {'word': 'Минюст', 'short': 'минюст', 'count': 3},
 {'word': 'СМИ-иноагентов', 'short': 'сми-иноагент', 'count': 3},
 {'word': 'физлиц-иноагентов', 'short': 'физлиц-иноагент', 'count': 3},
 {'word': 'Глуховского', 'short': 'глуховск', 'count': 2}]

In [2]:
from cfo_parser import get_cfo_days

data = get_cfo_days(30)

In [3]:
.gitconfig.gitconfig

124

'\n\n\r\n«Сбербанк» запустит собственную программу реструктуризации кредитов для малого и микробизнеса, сообщает ТАСС со ссылкой на слова заместителя председателя правления кредитной организации Анатолия Попова.\r\n\n\r\n «В случае мобилизации индивидуальных предпринимателей и собственников, доля участия которых в компании составляет более 50%, возможно получение отсрочки в погашении основного долга и процентов», – пояснил топ-менеджер.\r\n\n\r\n Заявку на эту услугу можно подать в интернет-банке для предпринимателей «СберБизнес», добавил он.\r\n\n\r\n Узнать больше о планах и опыте этой кредитной организации вы сможете 20-21 октября на Семнадцатом форуме «Внутренний и внешний электронный документооборот», где выступит Алексей Фролов, руководитель проектов «СберБанка».\r\n '

In [111]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:104.0) Gecko/20100101 Firefox/104.0',
    }
)


#"https://www.cfo-russia.ru/novosti/?article=73485"
def get_cfo_article(url):
    text = requests.get(url, headers=headers).text
    soup = BeautifulSoup(text, 'html.parser')
    import re
    CLEANR = re.compile('<.*?>') 

    def cleanhtml(raw_html):
        return re.sub(CLEANR, '', raw_html)


    html = str(soup.find("div", "news-detail").find("span", itemprop="description"))
    return cleanhtml(html).replace("\t", "").replace("\xa0", "")

# "https://www.cfo-russia.ru/novosti/?PAGEN_1=4"
def get_cfo_page(url):
    text = requests.get(url, headers=headers).text
    soup = BeautifulSoup(text, 'html.parser')

    res = []
    for i in soup.body.find(id="content").find("ul", "item-list").find_all("li"):
        title_a = i.find("div", "title").a
        url = "https://www.cfo-russia.ru" + str(title_a.get("href"))
        title = str(title_a.string).strip()
        description = str(i.find("div", "description").string).strip()
        date = str(i.find("div", "date-box").string).strip()
        timestamp = time.mktime(datetime.datetime.strptime(date, "%d.%m.%Y").timetuple())
        text = get_cfo_article(url)
        res.append({
            "url":url,
            "title": title,
            "description": description,
            "text": text,
            "timestamp": timestamp
        })
    return res
      
# get_cfo(0, 20)
def get_cfo(offset, count):
    res = []
    while len(res) < count:
        url = "https://www.cfo-russia.ru/novosti/?PAGEN_1=" + str(offset + 1)
        offset += 1
        res += get_cfo_page(url)
        
    return res[:count]

# get_cfo_days(10)
def get_cfo_days(days):
    res = []
    time_begin = time.time() - timedelta(days=days).total_seconds()
    offset = 0 # datetime.date.today()
    while len(res) == 0 or time_begin <= res[-1]["timestamp"]:
        url = "https://www.cfo-russia.ru/novosti/?PAGEN_1=" + str(offset + 1)
        offset += 1
        res += get_cfo_page(url)
    return list(filter(lambda i : time_begin <= i["timestamp"], res))


# get_cfo_page("https://www.cfo-russia.ru/novosti/?PAGEN_1=1")

[{'url': 'https://www.cfo-russia.ru/novosti/index.php?article=73759',
  'title': 'Минпромторг предложил освободить от акциза на жидкую сталь ещё 10 компаний',
  'description': 'Минпромторг предложил освободить от уплаты акциза на жидкую сталь еще 10 предприятий, в том числе «Уральскую сталь» (принадлежит Загорскому трубному заводу) и завод «Ижсталь» (входит в «Мечел»).',
  'text': '\n\n\r\nМинпромторг предложил освободить от уплаты акциза на жидкую сталь еще 10 предприятий, сообщает «Прайм» со ссылкой на опубликованный проект приказа ведомства.\r\n\n\r\n Список планируется расширить следующими предприятиями:\r\n\n\r\n · Златоустовский металлургический завод (Челябинская область);\r\n\n\r\n · «Ижсталь» (входит в «Мечел»);\r\n\n\r\n · «НПО Лаборатория специальных сталей и сплавов» (Санкт-Петербург, входит в машиностроительную группу «Кировский завод»);\r\n\n\r\n · «Уральскую сталь» (принадлежит Загорскому трубному заводу);\r\n\n\r\n · «Петросталь» (Санкт-Петербург, входит в машиностроите

In [112]:
len(_)

18

datetime.date(2022, 10, 7)

In [95]:
len(_)

20